You may find this series of notebooks at https://github.com/databricks-industry-solutions/jsl-financial-nlp

In [0]:
%pip install johnsnowlabs==4.2.3 networkx==2.5 decorator==5.0.9 plotly==5.1.0 

Python interpreter will be restarted.
Collecting johnsnowlabs==4.2.3
 Downloading johnsnowlabs-4.2.3-py3-none-any.whl (74 kB)
Requirement already satisfied: networkx==2.5 in /databricks/python3/lib/python3.8/site-packages (2.5)
Collecting decorator==5.0.9
 Downloading decorator-5.0.9-py3-none-any.whl (8.9 kB)
Requirement already satisfied: plotly==5.1.0 in /databricks/python3/lib/python3.8/site-packages (5.1.0)
Requirement already satisfied: nlu==4.0.1rc4 in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (4.0.1rc4)
Requirement already satisfied: spark-nlp==4.2.4 in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (4.2.4)
Requirement already satisfied: databricks-api in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (0.8.0)
Requirement already satisfied: colorama in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (0.4.6)
Requirement already satisfied: spark-nlp-display==4.1 in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (4.1)
Collecting pyspark==3.1.2
 Downloading pyspark-3.1.2.tar.gz (212.4 MB)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (2.25.1)
Requirement already satisfied: dataclasses in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (0.6)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (1.21.6)
Requirement already satisfied: pydantic in /databricks/python3/lib/python3.8/site-packages (from johnsnowlabs==4.2.3) (1.8.2)
Requirement already satisfied: tenacity>=6.2.0 in /databricks/python3/lib/python3.8/site-packages (from plotly==5.1.0) (6.2.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from plotly==5.1.0) (1.15.0)
Requirement already satisfied: pandas>=1.3.5 in /databricks/python3/lib/python3.8/site-packages (from nlu==4.0.1rc4->johnsnowlabs==4.2.3) (1.5.2)
Requirement already satisfied: pyarrow>=0.16.0 in /databricks/python3/lib/python3.8/site-packages (from nlu==4.0.1rc4->johnsnowlabs==4.2.3) (4.0.0)
Collecting py4j==0.10.9
 Downloading py4j-0.10.9-py2.py3-none-any.whl (198 kB)
Requirement already satisfied: svgwrite==1.4 in /databricks/python3/lib/python3.8/site-packages (from spark-nlp-display==4.1->johnsnowlabs==4.2.3) (1.4)
Requirement already satisfied: ipython in /databricks/python3/lib/python3.8/site-packages (from spark-nlp-display==4.1->johnsnowlabs==4.2.3) (7.22.0)
Requirement already satisfied: python-dateutil>=2.8.1 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.3.5->nlu==4.0.1rc4->johnsnowlabs==4.2.3) (2.8.1)
Requirement already satisfied: pytz>=2020.1 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.3.5->nlu==4.0.1rc4->johnsnowlabs==4.2.3) (2020.5)
Requirement already satisfied: databricks-cli in /databricks/python3/lib/python3.8/site-packages (from databricks-api->johnsnowlabs==4.2.3) (0.14.3)
Requirement already satisfied: click>=6.7 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli->databricks-api->johnsnowlabs==4.2.3) (7.1.2)
Requirement already satisfied: tabulate>=0.7.7 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli->databricks-api->johnsnowlabs==4.2.3) (0.8.7)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests->johnsnowlabs==4.2.3) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests->johnsnowlabs==4.2.3) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests->johnsnowlabs==4.2.3) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests->johnsnowlabs==4.2.3) (4.0.0)
Requirement already satisfied: pygmen

# Introduction
In this series of notebooks, we are going to analyze a 10K filing, obtained from US Security Exchange Commission's Edgar database, and create a financial knowledge graph with information from it, including registry information...
- `Official name of the company used in Edgar`
- `Other identification numbers, as CIK, SIC (Sector Code), IRS`
- `Stock information: Stock Market, Title Class, Class values, Trading Symbol, etc.`
- `Registry information (addresses, phone numbers, state, etc)`

... information about **other companies**...
- `Competitors`
- `Companies in the Supply Chain / mentioned in the filing to be in the ecosystem of the company`

... **people** ...
- `Current C-level managers`
- `Past C-level managers mentioned in the 10K filing (usually founders and co-founders)`
- `and their past experiences, if mentioned!`

Also, we will apply **Data Augmentation** using **offline John Snow Labs data sources** (`ChunkMappers`) to map the name of the company to information we know about it and John Snow Labs updates at a quartely basis, including:
- `Former Names`
- `Year of the change`

<img src="https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/solution_accelerator_ecosystem/series_of_notebooks.png" width="1000">

The result will be a unique Graph with nodes and relations containing the previously mentioned information:

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/solution_accelerator_ecosystem/img10.png" alt="drawing" width="800"/>

## Setup

The Financial NLP library already comes preinstalled with John Snow Labs - Databricks Partner Connect, available [here](https://docs.databricks.com/partners/ml/john-snow-labs.html).

Let's check `johnsnowlabs` library is installed

In [0]:
import johnsnowlabs
print(f"Spark NLP Licensed: {johnsnowlabs.settings.raw_version_jsl_lib}")
print(f"Spark NLP Open Source: {johnsnowlabs.settings.raw_version_nlp}")
print(f"Spark NLP PySpark: {johnsnowlabs.settings.raw_version_pyspark}")

Spark NLP Licensed: 4.2.3
Spark NLP Open Source: 4.2.4
Spark NLP PySpark: 3.1.2

Let's install the graph and visualization libraries

In [0]:
!pip install -q networkx==2.5 decorator==5.0.9 plotly==5.1.0

WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5aece422-a59f-439e-a0fb-4fb073ce15a5/bin/python -m pip install --upgrade pip' command.

Let's import them and reload, in case other versions of the libraries were present

In [0]:
import importlib
import networkx as nx
import decorator as dc
import plotly

importlib.reload(nx)
importlib.reload(dc)

Out[3]: <module 'decorator' from '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5aece422-a59f-439e-a0fb-4fb073ce15a5/lib/python3.8/site-packages/decorator.py'>

In [0]:
print(f"NetworkX version: {nx.__version__}")
print(f"Decorator version: {dc.__version__}")
print(f"Plotly version: {plotly.__version__}")

NetworkX version: 2.5
Decorator version: 5.0.9
Plotly version: 5.1.0

# You can proceed to 02 - Analysis!